In [3]:
cd '/content/drive/MyDrive/time series forcasting'

/content/drive/MyDrive/time series forcasting


In [4]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install "statsmodels==0.11.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.7 MB 7.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [6]:
# grid search ARIMA parameters for time series
import warnings
from math import sqrt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.6)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
	return best_cfg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


In [ ]:
import numpy as np
from openpyxl import load_workbook
from matplotlib import pyplot
workbook = load_workbook(filename='traffic_income_92_1400_Tehran (2).xlsx')
sheets = workbook.sheetnames

import xlsxwriter
Result_traffic = xlsxwriter.Workbook('Result_Final_traffic.xlsx')
worksheet = Result_traffic.add_worksheet()

service =[3,4,5,6,7,8,10]
for i in range(32):
  for j in service:
    Dataset = []
    for k in range(9):
      Dataset.append(workbook[sheets[k]].cell(row=i+3, column=j).value)
    X = np.array(Dataset)
    # evaluate parameters
    p_values = [0, 1, 2, 4, 6, 8, 10, 12]
    d_values = range(0, 3)
    q_values = range(0, 3)
    warnings.filterwarnings("ignore")
    best_cfg = evaluate_models(X, p_values, d_values, q_values)
    ###############################################################
    train_size = int(len(X) * 0.6)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    predictions = list()
    # walk-forward validation
    try:
      for t in range(len(test)):
        model = ARIMA(history, order=best_cfg)
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
      # evaluate forecasts
      rmse = sqrt(mean_squared_error(test, predictions))
      print('Test RMSE: %.3f' % rmse)
      # plot forecasts against actual outcomes
      pyplot.plot(test)
      pyplot.plot(predictions, color='red')
      pyplot.show()
      model = ARIMA(history, order=best_cfg)
      try:   
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        worksheet.write(i, service.index(j), predictions[-1])
        print("The one step-ahead prediction is:",predictions[-1])
      except:
        continue
    except:
      continue
Result_traffic.close()

ARIMA(0, 0, 0) RMSE=1492439.998
ARIMA(0, 0, 1) RMSE=1249617.831
ARIMA(0, 1, 0) RMSE=932340.595
ARIMA(0, 2, 0) RMSE=540920.803
ARIMA(0, 2, 2) RMSE=529601.939
ARIMA(1, 0, 0) RMSE=1232405.956
ARIMA(1, 0, 1) RMSE=1491990.675
ARIMA(1, 1, 0) RMSE=883835.068
ARIMA(1, 2, 0) RMSE=535853.470
ARIMA(1, 2, 2) RMSE=547320.072
ARIMA(2, 0, 0) RMSE=919393.642
ARIMA(2, 0, 1) RMSE=1192499.550
ARIMA(2, 1, 0) RMSE=912354.337
ARIMA(2, 2, 2) RMSE=548767.701
ARIMA(4, 2, 0) RMSE=624678.289
ARIMA(4, 2, 2) RMSE=587843.649
ARIMA(6, 0, 0) RMSE=698023.309
ARIMA(6, 2, 0) RMSE=698845.326
ARIMA(6, 2, 1) RMSE=731929.365
ARIMA(6, 2, 2) RMSE=732618.187
ARIMA(8, 0, 0) RMSE=1298563.924
ARIMA(8, 0, 1) RMSE=1310454.241
ARIMA(8, 0, 2) RMSE=1322771.264
ARIMA(8, 1, 0) RMSE=912528.029
ARIMA(8, 1, 1) RMSE=913951.923
ARIMA(8, 1, 2) RMSE=915755.929
ARIMA(8, 2, 0) RMSE=623927.555
ARIMA(8, 2, 1) RMSE=630378.325
ARIMA(8, 2, 2) RMSE=636107.743
ARIMA(10, 0, 0) RMSE=1298563.924
ARIMA(10, 0, 1) RMSE=1310454.278
ARIMA(10, 0, 2) RMSE=132276